# BB84 protocol

This notebook illustrates a simplified implementation of the [BB84](https://en.wikipedia.org/wiki/BB84) protocol using the [qiskit](https://qiskit.org/) framework.
The steps involved can be summarized as follows:

1. preparation of quantum states (encoding)
2. measures of the states (decoding)
3. sifting
4. parameter estimation

For each step, a class performing the required operations has been implemented.

Let's start with the necessary imports:

In [1]:
import sys  
sys.path.insert(0, '../src')

In [2]:
from bb84.encoding import Encoder, EncodingCircuit
from bb84.decoding import Decoder, DecodingCircuit
from bb84.sifting import Sifter
from bb84.estimation import Estimator
from random import choices
from bitstring import Bits
from utils import Sampler, add_noise

In [3]:
length = 5000 # use a smaller value to speed up the computation

## 1. Preparation of quantum states
Alice generates two random bit strings, `a_raw_key` and `a_bases` each of length `length`

In [4]:
a_raw_key = Bits(choices((0,1), k=length))
a_bases = choices((0,1), k=length)
#print(f'a_raw_key : {a_raw_key.bin}') # 01001...
#print(f'a_bases   : {a_bases}')   # [1,0,1,0,1,...]

She then prepares `length` qubits in one of four states, according to her random strings:   

$|0>$ if `a_raw_key[i] == 0` and `a_bases[i] == 0`   
$|1>$ if `a_raw_key[i] == 1` and `a_bases[i] == 0`   
$\frac{|0> + |1>}{\sqrt{2}}$ if `a_raw_key[i] == 0` and `a_bases[i] == 1`   
$\frac{|0> + |1>}{\sqrt{2}}$ if `a_raw_key[i] == 1` and `a_bases[i] == 1`      

The states are generated using the following circuits:

In [5]:
EncodingCircuit(0,0).circuit.draw('text') # a_raw_key[i] == 0 and a_bases[i] == 0

░ 
q: ─░─
    ░

In [6]:
EncodingCircuit(1,0).circuit.draw('text') # a_raw_key[i] == 1 and a_bases[i] == 0

┌───┐ ░ 
q: ┤ X ├─░─
   └───┘ ░

In [7]:
EncodingCircuit(0,1).circuit.draw('text') # a_raw_key[i] == 1 and a_bases[i] == 0

┌───┐ ░ 
q: ┤ H ├─░─
   └───┘ ░

In [8]:
EncodingCircuit(1,1).circuit.draw('text') # a_raw_key[i] == 1 and a_bases[i] == 1

┌───┐┌───┐ ░ 
q: ┤ X ├┤ H ├─░─
   └───┘└───┘ ░

The class to which is delegated the creation of these states is `bb84.encoding.Encoder`:

In [9]:
encoder = Encoder()
states = encoder.encode(a_raw_key, a_bases)

Since to store `lenght` actual state vectors, $2^\text{lenght}$ float are required, `states` contains the circuits that produce that states

## 2. Measures of the states
Let's imagine that Alice transmitted the qubits she has prepared to Bob.  
Bob chooses `length` random bases, among the Z and X bases, in which he measures the qubits:

In [10]:
b_bases = choices((0,1), k=length)
#print(f'b_bases : {b_bases}') # [1,0,1,0,1,...]

Bob then measure the qubits using the chosen bases.   

Since in qiskit is possible to measure only in the computational basis, a measure operation is preceded by an appropriate unitary transformation.  

In this case, if `b_bases[i] == 1` the Hadamard gate is applied; otherwise, no transformation is needed

In [11]:
DecodingCircuit(1).circuit.draw('text')

┌───┐ ░ ┌─┐
        q: ┤ H ├─░─┤M├
           └───┘ ░ └╥┘
key_bit: 1/═════════╩═
                    0

In [12]:
DecodingCircuit(0).circuit.draw('text')

░ ┌─┐
        q: ─░─┤M├
            ░ └╥┘
key_bit: 1/════╩═
               0

The class to which is delegated the operations involved in measuring each of the `length` qubits is `bb84.decoding.Decoder`  
The result of `decode` operation is Bob's raw key:

In [13]:
decoder = Decoder()
b_raw_key = decoder.decode(b_bases, states)
#print(f'b_raw_key : {b_raw_key.bin}') # 01101...

## 3. Sifting
Let's imagine that Alice transmitted the list of her chosen bases to Bob, and vice versa.
They can now generate the sifted keys, starting from their raw keys, and using the bases where `a_bases[i] == b_bases[i]`   

The sifting is done using the class `bb84.sifting.Sifter`

Since this simplified implementation of the protocol is ideal, Alice's and Bob's sifted key will be the same; furthermore, the ratio of raw and sifted keys's  lengths will be close to $\frac{1}{2}$

In [14]:
sifter = Sifter(a_bases, b_bases)
a_sifted_key = sifter.sift(a_raw_key)
b_sifted_key = sifter.sift(b_raw_key)
#print(f'a_sifted_key : {a_sifted_key.bin}')
#print(f'b_sifted_key : {b_sifted_key.bin}')
print(f'Are the keys identical? {a_sifted_key == b_sifted_key}')
print(f'sifted_key/raw_key lengths ratio: {len(a_sifted_key)/len(a_raw_key)} (expected: {1/2})')

Are the keys identical? True
sifted_key/raw_key lengths ratio: 0.4892 (expected: 0.5)


## 3. Parameter estimation
Alice and Bob now have to make sure thate an eavesdropper, listening to their public communication, has not acquired information about the keys.   

They do so verifying that the error rate (the fraction of bits that differs in the two sifted keys) is below a certain threshold.

To estimate the error rate, they first extract a random sample from their sifted key (image that Alice tells Bob the position of the bits she used to extract the sample):

In [15]:
a_sampler = Sampler(a_sifted_key)
sampling_bitvector = a_sampler.sampling_bitvector # Alice tells Bob wich bits she used to extract the sample
# adding some noise to the sifted key, otherwise the error rate would be 0
b_sampler = Sampler(add_noise(b_sifted_key,0.1), sampling_bitvector)

a_sample = a_sampler.sample()
b_sample = b_sampler.sample()

#print(f'a_sample : {a_sample.bin}')
#print(f'b_sample : {b_sample.bin}')

Let's imagine that Alice and Bob transmitted their own sample to the other.
They can now estimate the error rate:

In [16]:
estimator = Estimator()
error_rate = estimator.estimate(a_sample, b_sample)
print(f'error_rate: {error_rate}')

error_rate: 0.11120196238757155


The error rate on the bits reamaining from the sample should be very close to the estimated one;
from this remaining keys, they can distillate a pair of shared secure keys performing error correction and privacy amplification

In [17]:
a_remaining = a_sampler.remaining()
b_remaining = b_sampler.remaining()
error_rate = estimator.estimate(a_remaining, b_remaining)
print(f'error_rate: {error_rate}')

error_rate: 0.08176614881439084
